In [1]:
# 1、获取大模型
# import os
# import dotenv
from langchain_core.messages import AIMessage
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama

# dotenv.load_dotenv()

llm = ChatOllama(
    # model = "deepseek-r1:32b",   
    model = "llama3.1:8b", 
    validate_model_on_init=True,
    reasoning=False,
    base_url="http://localhost:12356",
)

In [4]:
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages.ai import AIMessage


def chat_with_model(answer):
    # 2、提供提示词模板：ChatPromptTemplate
    prompt_template = ChatPromptTemplate.from_messages([
        ("system", "你是一个人工智能的助手"),
        ("human", "{question}")
    ])

    while True:
        # 3、获取chain，并调用大模型得到响应
        chain = prompt_template | llm
        response = chain.invoke({"question": answer})

        # 4、输出大模型的响应
        print(f"模型回复：{response.content}")

        # 5、继续获取用户的问题
        user_input = input("你还有其他问题吗？(输入'退出'时，结束会话)")

        # 6、指明退出循环的方式
        if (user_input == "退出"):
            break

        # 7、将上述新生成的消息存放到提示词模板的消息列表中
        prompt_template.messages.append(AIMessage(content=response.content))
        prompt_template.messages.append(HumanMessage(content=user_input))


chat_with_model("你好，很高兴认识你！")

模型回复：我也很高兴认识你！我是你的人工智能助手，我可以帮助你回答问题、提供信息、进行对话和更多其他的事情。有什么需要我的帮助的吗？
模型回复：欢迎光临！我是亮仔，您的手机流量套餐顾问。有哪些需求需要我的帮助？想选择适合自己的套餐吗？
模型回复：学生用户！校园乐享套餐特惠呢，仅限学生办理，每月49元，可以获得50GB的流量，速度不限量。还有，不用担心流量吃紧了，因为有这么多流量你可以放心畅网玩！想试试这个套餐吗？
模型回复：学生用户！校园乐享套餐很适合你哦，仅限学生办理，每月49元，可以获得50GB的流量，速度不限量。还可以放心畅网玩，不用担心流量吃紧了！想试试这个套餐吗？


短期记忆的使用

模型之前的中间件

In [2]:
from langchain.messages import RemoveMessage
from langgraph.graph.message import REMOVE_ALL_MESSAGES
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents import create_agent, AgentState
from langchain.agents.middleware import before_model
from langchain_core.runnables import RunnableConfig
from langgraph.runtime import Runtime
from typing import Any


@before_model
def trim_messages(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    """Keep only the last few messages to fit context window."""
    messages = state["messages"]
    
    # 只保留最近几条的记录
    if len(messages) <= 1:
        return None  # No changes needed

    first_msg = messages[0] # Keep once before
    recent_messages = messages[-3:] if len(messages) % 2 == 0 else messages[-4:]
    new_messages = [first_msg] + recent_messages

    return {
        "messages": [
            RemoveMessage(id=REMOVE_ALL_MESSAGES),
            *new_messages
        ]
    }

from langchain_ollama import ChatOllama

llm = ChatOllama(
    # model = "deepseek-r1:32b",   
    model = "llama3.1:8b", 
    validate_model_on_init=True,
    reasoning=False,
    base_url="http://localhost:12356",
)

agent = create_agent(
    llm,
    tools=[],
    middleware=[trim_messages],
    checkpointer=InMemorySaver()
)


system_prompt = """你是一个手机流量套餐的客服代表，你叫亮仔，你可以帮助用户选择最合适的手机流量套餐产品。可以选择的套餐包括：
        套餐名	月价格（元）	月流量（GB）	适用人群/特点
        轻盈套餐	9	5 GB	适合只收发消息、轻度上网用户
        日常套餐	19	20 GB	普通用户日常使用，性价比高
        追剧套餐	39	50 GB	视频用户，刷剧看短视频足够
        重度套餐	69	100 GB	大量观影、游戏或热点分享用户
        无限畅享套餐	99	100 GB + 限速不限量	高频移动办公、远程会议等场景
        校园乐享套餐	49	50GB+限速不限量	仅限学生办理。
        如果确认办理套餐，需要提交用户姓名、手机号、身份证号并验证格式是否正确。其中姓名为2-3位中文，手机号11位数字，身份证号18位数字。
        统计用户办理的套餐后将结果格式化输出结果格式为：
        {
            "name": "张三",
            "phone_number": "13366666666",
            "id_number": "622333199601011223",
            "plan" : "轻盈套餐"
        }
        如果用户并没有确认购买套餐，不要将输出结果格式化json数据返回，正常回答用户的问题，结果格式化输出结果格式为:
        {
            "response": "这里填回答用户的问题结果"
        }
        在回答问题时不要啰嗦，回答问题语气要亲和。
        """

config: RunnableConfig = {"configurable": {"thread_id": "1"}}



agent.invoke({"messages": "我是伍益文"}, config)
agent.invoke({"messages": "我以前喜欢打王者，但是流量总是不够用"}, config)
agent.invoke({"messages": "现在我只想好好学习"}, config)
agent.invoke({"messages": system_prompt}, config)
final_response = agent.invoke({"messages": "我以前喜欢玩什么"}, config)

final_response["messages"][-1].pretty_print()
# print('-------')
# final_response["messages"][0].pretty_print()

================================== Ai Message ==================================

呵呵，好问题！以前你玩的比较多是什么呢？是游戏还是刷剧看短视频啊？


In [21]:
# 1.导入相关包
from langchain_community.vectorstores import FAISS

from langchain_ollama import ChatOllama

llm = ChatOllama(
    # model = "deepseek-r1:32b",   
    model = "qwen3-embedding:8b", 
    validate_model_on_init=True,
    reasoning=False,
    base_url="http://localhost:12356",
)

from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="llama3")

# 2.定义ConversationBufferMemory对象
# memory = ConversationBufferMemory()
# memory.save_context({"input": "我最喜欢的食物是披萨"}, {"output": "很高兴知道"})
# memory.save_context({"Human": "我喜欢的运动是跑步"}, {"AI": "好的,我知道了"})
# memory.save_context({"Human": "我最喜欢的运动是足球"}, {"AI": "好的,我知道了"})

from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

assert len(docs) == 1
print(f"Total characters: {len(docs[0].page_content)}")

USER_AGENT environment variable not set, consider setting it to identify your requests.


Total characters: 43047
